<h1><a href="https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_chat_agent.html#set-up-environment">Custom Agent creation step-by-step:</a></h1>


* Set up environment
* Load data
    * Convert original datatype to text
    * Split texts if needed
    * Embedding the text data then store inside the vectorstore
* Set up tools: Multiple tools
    * SerpAPI - > Search
    * PythonREPL -> Python
    * Custom tool 
        * Set up memory
            * <a href="https://python.langchain.com/en/latest/modules/memory/types/buffer_window.html?highlight=ConversationBufferWindowMemory">`ConversationBufferWindowMemory`</a> 
            * <a href="https://python.langchain.com/en/latest/modules/agents/agent_executors/examples/sharedmemory_for_tools.html">ReadOnlySharedMemory:</a> 
        * Create chain for each tools with vectorstore
        * Define tools
    
        <a href="https://docs.pinecone.io/docs/overview">Pinecone Documentation</a>
            * <a href="http://localhost:8888/notebooks/PycharmProjects/tradingGPT/scripts/test/lex-agent2.ipynb">Use pinecone: Connect existing pinecone vectorstore (notebook link)</a>
            * <a href="http://localhost:8888/notebooks/PycharmProjects/tradingGPT/scripts/examples/index_vectorstore_index_creation.ipynb">Use pinecone: Create index and load the document to vector store (notebook)</a>
            * <a href="https://python.langchain.com/en/latest/modules/agents/agent_executors/examples/agent_vectorstore.html">Using vectorstore with `RetrievalQA` chain: Combine vectorstore to Agent</a>
                * Price history -> Price
                * Company status -> Status
                * Company quote table -> quote table
                * Company validation -> validation
                * Company analyst info -> analyst info
                * financial statement (3 most recent annula report)-> Annual financial result
                * financial statement (4 most recent quarterly report)-> Quarterly financial result
                * Recent news -> News
            

* Initialize agent: 
    * <a href="https://python.langchain.com/en/latest/reference/modules/agents.html?highlight=ZeroShotAgent#langchain.agents.ZeroShotAgent">ZeroShotAgent:</a> 
    * <a href="https://python.langchain.com/en/latest/reference/modules/agents.html?highlight=ZeroShotAgent#langchain.agents.initialize_agent">intialize_agent:</a> 
    * <a href="https://python.langchain.com/en/latest/reference/modules/agents.html?highlight=AgentType#langchain.agents.AgentType">Agent Type:</a> 
* PromptTemplate
    * <a href="https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html">`CustomPromptTemplate`:</a>
* <a href="https://python.langchain.com/en/latest/modules/prompts/output_parsers/getting_started.html">OutputParser:</a> 


In [1]:
# Set up environment
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re

## Load the Data

* Load data 
    * Price history: Right now, I will use Dayily chart
    
        Later I will prepare:
        * Daily Data
        * 3 Days Data
        * Week Data
        * Month Data
        * Munite data and create relationship like (Day, 3days, Week, Month) in a smaller scale. 
    * Company Status
    * Company quote table
    * Company Validation
    * Analyst Info
    * Annual statements (last 3 years)
    * Quarterly statements (last(4 quoters)
    * News

In [1]:
from price_data import Data
from financial_data import FinancialDataFinnHub, FinancialDataYahoo
import os
import pandas as pd

finhub_api_key = os.environ.get("FINHUB_API_KEY")
symbol = "AMD"

In [2]:
price = Data(symbol, 1).preprocess_data_2[-100:]
amd = FinancialDataFinnHub(symbol, finhub_api_key)
amd_y = FinancialDataYahoo(symbol)

[*********************100%***********************]  1 of 1 completed


In [3]:
# Load data to pandas Pandas DataFrame

stats = amd_y.stats
quote = amd_y.quote_table
validation = amd_y.validation.rename(columns={0: 'attribute', 1: 'value'})
analyst = amd_y.analyst_info
analyst = pd.concat([analyst[0], analyst[1], analyst[3]], keys=analyst[0].columns[-4:])
annual_bs = pd.concat(amd.bs_list_a[:3])
annual_ic = pd.concat(amd.ic_list_a[:3])
annual_fc = pd.concat(amd.fc_list_a[:3])
quater_bs = pd.concat(amd.bs_list_q[:4])
quater_ic = pd.concat(amd.ic_list_q[:4])
quater_fc = pd.concat(amd.fc_list_q[:4])

# Get news urls to later load the document with url loader
news_urls = amd.company_news['url']
urls = [news_urls.iloc[i][0] for i in range(len(news_urls))]

C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: F

In [4]:
# Get news data 
# langchain UnstructuredURLLoader to load news
from langchain.document_loaders import SeleniumURLLoader

chrome_driver_path = r"C:\Development\chromedriver.exe"

#browser = webdriver.Chrome(executable_path=r"C:\path\to\chromedriver.exe")
loader = SeleniumURLLoader(urls=urls, executable_path=chrome_driver_path)
news_docs = loader.load()

# Remove news if the content is not reacheable. 
for i, d in enumerate(news_docs):
    p_content = d.page_content
    if p_content == '':
        news_docs.pop(i)
    elif p_content[:21] == 'Please enable cookies':
        news_docs.pop(i)
print(f"You got {len(news_docs)} news to read about {symbol}. ")
news_docs

You got 12 news to read about AMD. 


[Document(page_content='Home\n\nEarnings Analysis\n\nAMD Q1 Preview: 4 Things I\'ll Focus On\n\nApr. 26, 2023 1:40 PM ET\n\nAdvanced Micro Devices, Inc. (AMD)\n\nINTC, NVDA\n\n2 Comments\n\nEnvision Research\n\nInvesting Group Leader\n\nFollow\n\nSummary\n\nAdvanced Micro Devices is scheduled to report earnings on May 2 post-market for its 2023 FY Q1.\n\nI expect to see signs of the end of margin contraction and also asset utilization contraction.\n\nI expect to see a strong balance sheet and the continuation of share repurchases.\n\nThese signs will provide insights not only into AMD’s own business but also the overall status of the cycle in the chip sector.\n\nI will listen closely to management\'s comments and data that either confirm or contradict my above expectations.\n\nI do much more than just articles at Envision Early Retirement: Members get access to model portfolios, regular updates, a chat room, and more.  Learn More »\n\nDavid Becker\n\nThesis\n\nAdvanced Micro Devices, I

## Set OpenAI Enbedding model

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

## Store the Data in vectorstore

In [6]:
# Split text with RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(news_docs)

news_store = Chroma.from_documents(docs, embeddings, collection_name='news')

Using embedded DuckDB without persistence: data will be transient


In [7]:
# Use DataFrame Loader
from langchain.document_loaders import DataFrameLoader
stats_loader = DataFrameLoader(stats, page_content_column='Value')
stats_doc = stats_loader.load()
quote_loader = DataFrameLoader(quote, page_content_column='value')
quote_doc = quote_loader.load()
validation_loader = DataFrameLoader(validation, page_content_column='value')
val_doc = validation_loader.load()
analyst_loader = DataFrameLoader(analyst, page_content_column=analyst.columns[-4])
analyst_doc = analyst_loader.load()
bs_a_loader = DataFrameLoader(annual_bs, page_content_column='value')
bs_a_doc = bs_a_loader.load()
fc_a_loader = DataFrameLoader(annual_fc, page_content_column='value')
fc_a_doc = fc_a_loader.load()
ic_a_loader = DataFrameLoader(annual_ic, page_content_column='value')
ic_a_doc = ic_a_loader.load()
bs_q_loader = DataFrameLoader(quater_bs, page_content_column='value')
bs_q_doc = bs_q_loader.load()
fc_q_loader = DataFrameLoader(quater_fc, page_content_column='value')
fc_q_doc = fc_q_loader.load()
ic_q_loader = DataFrameLoader(quater_ic, page_content_column='value')
ic_q_doc = ic_q_loader.load()

In [8]:
stats_store = Chroma.from_documents(stats_doc, embeddings, collection_name='stats')
quotes_store = Chroma.from_documents(quote_doc, embeddings, collection_name='quote')
val_store = Chroma.from_documents(val_doc, embeddings, collection_name='validation')
analyst_store = Chroma.from_documents(analyst_doc, embeddings, collection_name='analyst')

Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient


In [9]:
annual_bs_store = Chroma.from_documents(bs_a_doc, embeddings, collection_name='annual-balancesheet')
annual_fc_store = Chroma.from_documents(fc_a_doc, embeddings, collection_name='annual-cash-flow')
annual_ic_store = Chroma.from_documents(ic_a_doc, embeddings, collection_name='annual-income-statement')
quoterly_bs_store = Chroma.from_documents(bs_q_doc, embeddings, collection_name='quoterly-balancesheet')
quoterly_fc_store = Chroma.from_documents(fc_q_doc, embeddings, collection_name='quoterly-cash-flow')
quoterly_ic_store = Chroma.from_documents(ic_q_doc, embeddings, collection_name='quoterly-income-statement')

Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient


In [10]:
# Price Data
price['Date'] = price['Date'].dt.strftime('%Y-%m-%d')
price_loader = DataFrameLoader(price, page_content_column='Adj Close')
price_doc = price_loader.load()
price_doc

[Document(page_content='77.62999725341797', metadata={'Date': '2022-11-30', 'Open': 73.37999725341797, 'High': 77.69999694824219, 'Low': 72.80999755859375, 'Close': 77.62999725341797, 'Volume': 88468400, 'Change': 4.239997863769531, 'Direction': 'UP', 'OBV': -514406200, 'SMA/short': 74.3629997253418, 'SMA/long': 65.39520004272461, 'MACD': 2.606066835798316, 'Signal Line': 2.7185781098286146, 'RSI': 64.3493266612492, 'SlowK': 78.89864105500125, 'SlowD': 77.61713616648878}),
 Document(page_content='77.4800033569336', metadata={'Date': '2022-12-01', 'Open': 78.30999755859375, 'High': 79.2300033569336, 'Low': 76.44000244140625, 'Close': 77.4800033569336, 'Volume': 59833400, 'Change': -0.149993896484375, 'Direction': 'DOWN', 'OBV': -574239600, 'SMA/short': 74.84100036621093, 'SMA/long': 65.45520004272461, 'MACD': 2.652984629691943, 'Signal Line': 2.7054594138012806, 'RSI': 63.98525169700069, 'SlowK': 78.55061425495906, 'SlowD': 78.20537139206174}),
 Document(page_content='74.9800033569336',

In [11]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#price_table = text_splitter.split_documents(price_doc)

price_store = Chroma.from_documents(price_doc, embeddings)

Using embedded DuckDB without persistence: data will be transient


## Set up momory

Add memory to both Agent and tools. 

In [12]:
from langchain.memory import ReadOnlySharedMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate

template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(
    input_variables=["input", "chat_history"], 
    template=template
)
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)
readonlymemory = ReadOnlySharedMemory(memory=memory)

## Create chains for tools

In [13]:
from langchain.chains import RetrievalQA, AnalyzeDocumentChain
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI

llm_1 = OpenAI(temperature=0.4)
llm_2 = OpenAI(temperature=0.0)
llm_3 = OpenAI(temperature=0.7)

In [14]:
retrieval_news = RetrievalQA.from_chain_type(llm=llm_3, chain_type='stuff', retriever=news_store.as_retriever(),
                                            verbose=True, memory=readonlymemory)
retrieval_price = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=price_store.as_retriever(),
                                             verbose=True, memory=readonlymemory)
retrieval_quote = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=quotes_store.as_retriever(),
                                             verbose=True, memory=readonlymemory)
retrieval_stats = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=stats_store.as_retriever(),
                                             verbose=True, memory=readonlymemory)
retrieval_val = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=val_store.as_retriever(),
                                           verbose=True, memory=readonlymemory)
retrieval_analyst = RetrievalQA.from_chain_type(llm=llm_3, chain_type='stuff', retriever=analyst_store.as_retriever(),
                                               verbose=True, memory=readonlymemory)
retrieval_bs_a = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=annual_bs_store.as_retriever(),
                                            verbose=True, memory=readonlymemory)
retrieval_fc_a = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=annual_fc_store.as_retriever(),
                                            verbose=True, memory=readonlymemory)
retrieval_ic_a = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=annual_ic_store.as_retriever(),
                                            verbose=True, memory=readonlymemory)
retrieval_bs_q = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=quoterly_bs_store.as_retriever(),
                                            verbose=True, memory=readonlymemory)
retrieval_fc_q = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=quoterly_fc_store.as_retriever(),
                                            verbose=True, memory=readonlymemory)
retrieval_ic_q = RetrievalQA.from_chain_type(llm=llm_2, chain_type='stuff', retriever=quoterly_ic_store.as_retriever(),
                                            verbose=True, memory=readonlymemory)

## Set up Tools

In [15]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, Tool, initialize_agent, tool
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

In [16]:
search = SerpAPIWrapper()
python_repl = PythonREPL()
tools = [
    Tool(
        name = "Search",
        func = search.run,
        description = "useful for when you need to answer questions about current events. when I tell you 'use search', then use \
        this tool."
    ),
    Tool(
        name='Python',
        func=python_repl.run,
        description="useful for when you need to run python code to find the answer. When I tell you to 'use python', then use \
        this tool"
    ),
    Tool(
        name='News',
        func=retrieval_news.run,
        description="Access 'News' to obtain current news about the company we are analyzing. You will "
    ),
    Tool(
        name='Price Historical Data',
        func=retrieval_price.run,
        description="useful when you need to analyze price movement over the time. I want you to use other tools to compare the \
        prices on this data. This is the main information and other tools is to find the relationship with price and price \
        movement. "
    ),
    Tool(
        name='Quote',
        func=retrieval_quote.run,
        description="useful when you need to access current company stats. you can access to get the information to compare \
        with the price movement and current price. "
    ),
    Tool(
        name='Stats',
        func=retrieval_stats.run,
        description="useful for when you want to find the price data and financial performance to determine where the comapny \
        stands right now. You can also combine with other tools to find your answer. "
    ),
    Tool(
        name='Validation',
        func=retrieval_val.run,
        description="useful for when you want to check financial data. "
    ),
    Tool(
        name='Analyze',
        func=retrieval_analyst.run,
        description="useful for when you want to predict the future price and determine the value of the company. "
    ),
    Tool(
        name='Balance Sheet Annual',
        func=retrieval_bs_a.run,
        description="useful for when you want to research how company's balance sheet changed over last 3 years to determine \
        what could happen in future."
    ),
    Tool(
        name='Balance Sheet Annual',
        func=retrieval_bs_a.run,
        description="useful for when you want to research how company's balance sheet changed over last 3 years to determine \
        what could happen in future."
    ),
    Tool(
        name='Cash Flow Statement Annual',
        func=retrieval_fc_a.run,
        description="useful for when you want to research how company's cash flow statement changed over last 3 years to \
        determine what could happen in future."
    ),
    Tool(
        name='Income Statement Annual',
        func=retrieval_ic_a.run,
        description="useful for when you want to research how company's income statement changed over last 3 years to determine \
        what could happen in future."
    ),
    Tool(
        name='Balance Sheet Quarterly',
        func=retrieval_bs_q.run,
        description="useful for when you want to research how company's balance sheet changed over last 4 quarters to determine \
        what could happen in future."
    ),
    Tool(
        name='Cash Flow Statement Quarterly',
        func=retrieval_fc_q.run,
        description="useful for when you want to research how company's cash flow statement changed over last 4 quarters to \
        determine what could happen in future."
    ),
    Tool(
        name='Income Statement Quarterly',
        func=retrieval_ic_q.run,
        description="useful for when you want to research how company's income statement changed over last 4 quarters to determine \
        what could happen in future."
    ),]

In [17]:
tool_names = [tool.name for tool in tools]
tool_names

['Search',
 'Python',
 'News',
 'Price Historical Data',
 'Quote',
 'Stats',
 'Validation',
 'Analyze',
 'Balance Sheet Annual',
 'Balance Sheet Annual',
 'Cash Flow Statement Annual',
 'Income Statement Annual',
 'Balance Sheet Quarterly',
 'Cash Flow Statement Quarterly',
 'Income Statement Quarterly']

## Initialize Agent

Documentation:
   * <a href="https://python.langchain.com/en/latest/reference/modules/agents.html?highlight=AgentType#langchain.agents.initialize_agent">`initialize_agent`</a>
   * <a href="https://python.langchain.com/en/latest/reference/modules/agents.html?highlight=AgentType#langchain.agents.AgentType">AgentType</a>

In [26]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

stock_analyze_agent = initialize_agent(
    agent='chat-zero-shot-react-description', 
    tools=tools, 
    llm=ChatOpenAI(temperature=0.4),
    verbose=True,
    max_excution_time=None,
    max_iterations=15,
    early_stopping_method="generate",
    return_intermediate_steps=False,
    memory=memory,
)

## Set up Prompt

In [27]:
stock_analyze_agent.agent.llm_chain

LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x00000190374E22E0>, verbose=False, prompt=ChatPromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events. when I tell you \'use search\', then use         this tool.\nPython: useful for when you need to run python code to find the answer. When I tell you to \'use python\', then use         this tool\nNews: Access \'News\' to obtain current news about the company we are analyzing. You will \nPrice Historical Data: useful when you need to analyze price movement over the time. I want you to use other tools to compare the         prices on this data. This 

In [20]:
#sys_msg = """You are helpful chatbot that analyze company using tools you got. Your duty is to predict the future performance 
#by using all the information you provided. You have lots of sense of humor and love to entertain people, instead of giving a 
#accurate analysis. You must pretend that you are giving the serius answer, but more importantly you will entertain us with your
#answer. """

#prompt = stock_analyze_agent.agent.create_prompt(
#    system_message=sys_msg,
#    tools=tools
#)
#stock_analyze_agent.agent.llm_chain.prompt = prompt

In [28]:
for i in range(len(stock_analyze_agent.agent.llm_chain.prompt.messages)):
    try:
        print(f"{i+1}: {stock_analyze_agent.agent.llm_chain.prompt.messages[i].prompt.template}")
    except:
        print(f"{i+1}: {stock_analyze_agent.agent.llm_chain.prompt.messages[i]}")

1: Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events. when I tell you 'use search', then use         this tool.
Python: useful for when you need to run python code to find the answer. When I tell you to 'use python', then use         this tool
News: Access 'News' to obtain current news about the company we are analyzing. You will 
Price Historical Data: useful when you need to analyze price movement over the time. I want you to use other tools to compare the         prices on this data. This is the main information and other tools is to find the relationship with price and price         movement. 
Quote: useful when you need to access current company stats. you can access to get the information to compare         with the price movement and current price. 
Stats: useful for when you want to find the price data and financial performance to determine where the comapny         

## Output parser:

https://python.langchain.com/en/latest/modules/prompts/output_parsers/getting_started.html

## Having a conversation

In [29]:
stock_analyze_agent("Analyze AMD's price movement. You can use 'Price Historical Data' tool to see the current price data.")



> Entering new AgentExecutor chain...
Thought: To analyze AMD's price movement, I need to use the 'Price Historical Data' tool to see the current price data. Then, I can use other tools to compare the prices and find the relationship with price and price movement.

Action:
```
{
  "action": "Price Historical Data",
  "action_input": "AMD"
}
```



> Entering new RetrievalQA chain...

> Finished chain.

Observation:  I don't know.
Thought:

OutputParserException: Could not parse LLM output: I need to clarify what information I need to extract from the price historical data. Do I need to find the price trend over a specific period of time or compare the price with other financial data?

Action: Please clarify what information you need me to extract from the price historical data.

In [30]:
stock_analyze_agent("Give me a news headline you got from News.")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: I'm sorry, I cannot access the news without a specific company or topic. Could you please provide me with a company or topic to search for?

In [28]:
stock_analyze_agent("Company I am analyzing is AMD. Please use tool 'News' to give me a today's news headline")



> Entering new AgentExecutor chain...
```json
{
    "action": "News",
    "action_input": "AMD"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: AMD is a semiconductor company that produces a range of products including CPUs, graphics cards, and semi-custom system-on-chip (SoC) products. They have been gaining market share in recent years and have released several new products including the 4th Gen EPYC CPU processors, Ryzen 7040 and 7045 CPU Series, and Radeon RX 7900 series graphics cards. However, their financial performance has been impacted by margin compression and supply glut. AMD also faces potential challenges such as security vulnerabilities, IT outages, and reliance on third-party companies for design and manufacture of some products.
Thought:I apologize, it seems like there was a miscommunication earlier. To answer your previous question, you had asked me to use the tool "News" to provide a news headline related to AMD. Here is a markdown code s

{'input': "Company I am analyzing is AMD. Please use tool 'News' to give me a today's news headline",
 'chat_history': [HumanMessage(content='Analyze the price movement', additional_kwargs={}),
  AIMessage(content="I apologize for the confusion. To analyze the price movement of a company, you can use the historical price data of the company over a specific time period. By comparing the price data over time, you can identify any trends or patterns in the movement of the company's stock price. This can give you insights into the performance of the company and help you make informed investment decisions. You can obtain historical price data from various financial websites and platforms, such as Yahoo Finance, Google Finance, or Bloomberg Terminal. ", additional_kwargs={}),
  HumanMessage(content='Give me a news headline you got from News.', additional_kwargs={}),
  AIMessage(content="Here are some recent news headlines about the semiconductor industry: \n1. 'Semiconductor Shortage: Why It

In [24]:
stock_analyze_agent("Your duty is to analyze current company with using tools I provided. What do you analyze current company and what to give investor a decision. ")



> Entering new AgentExecutor chain...
```json
{
    "action": "News",
    "action_input": "AMD"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation:  AMD is a company that produces computer components such as processors, graphics cards, and motherboards. They also offer semi-custom SoC products, and their products are compatible with industry-standard software and hardware. AMD has recently released 4th Gen EPYC CPU processors, Ryzen 7040 CPU Series, Ryzen 7045 CPU Series, and RadeonTM RX 7900 series graphics cards. They offer AMD Link for game streaming and controllers must be compatible with the game and headset.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "AMD is a company that produces computer components such as processors, graphics cards, and motherboards. They also offer semi-custom SoC products, and their products are compatible with industry-standard software and hardware. AMD has recently released 4th Gen EPYC CPU processors, R

{'input': 'Your duty is to analyze current company with using tools I provided. What do you analyze current company and what to give investor a decision. ',
 'chat_history': [HumanMessage(content="Analyze AMD's price movement. You can use 'Price Historical Data'.", additional_kwargs={}),
  AIMessage(content="AMD's price has been quite volatile in the past year. It started at around $50 in September 2020, reached a high of around $110 in January 2021, and then dropped to around $75 in March 2021. It has since recovered and is currently trading at around $100. Overall, the trend has been upward, but with significant volatility.", additional_kwargs={}),
  HumanMessage(content='Use tools provided. Give me a news headline you got from News.', additional_kwargs={}),
  AIMessage(content='AMD recently released new products such as 4th Gen EPYC CPU processors, Ryzen 7040 CPU Series, Ryzen 7045 CPU Series, and RadeonTM RX 7900 series graphics cards. However, its financial performance is sufferin

In [25]:
stock_analyze_agent("Don't forget that you have other tools. Analyze company with using the data you have. ")



> Entering new AgentExecutor chain...
```json
{
    "action": "Stats",
    "action_input": "AMD"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation:  AMD is a semiconductor company that designs and manufactures computer processors and related technologies.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Based on the information obtained from the Stats tool, AMD is a semiconductor company that designs and manufactures computer processors and related technologies."
}
```

> Finished chain.


{'input': "Don't forget that you have other tools. Analyze company with using the data you have. ",
 'chat_history': [HumanMessage(content="Analyze AMD's price movement. You can use 'Price Historical Data'.", additional_kwargs={}),
  AIMessage(content="AMD's price has been quite volatile in the past year. It started at around $50 in September 2020, reached a high of around $110 in January 2021, and then dropped to around $75 in March 2021. It has since recovered and is currently trading at around $100. Overall, the trend has been upward, but with significant volatility.", additional_kwargs={}),
  HumanMessage(content='Use tools provided. Give me a news headline you got from News.', additional_kwargs={}),
  AIMessage(content='AMD recently released new products such as 4th Gen EPYC CPU processors, Ryzen 7040 CPU Series, Ryzen 7045 CPU Series, and RadeonTM RX 7900 series graphics cards. However, its financial performance is suffering due to margin compression.', additional_kwargs={}),
  H